In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATADIR = '/content/drive/MyDrive/flowers'
IMG_SIZE = 180
CATEGORIES = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [4]:
from tqdm import tqdm
data = []
def create_data():
  for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):
      try:
        img_array = cv2.imread(os.path.join(path, img))
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        data.append([new_array,class_num])
      except Exception as e:
        print("Error")


In [5]:
create_data()
print(len(data))

 82%|████████▏ | 865/1055 [00:19<00:01, 158.38it/s]

Error


 91%|█████████▏| 963/1055 [00:19<00:00, 156.36it/s]

Error


 96%|█████████▋| 1018/1055 [00:19<00:00, 170.21it/s]

Error


100%|██████████| 984/984 [00:20<00:00, 48.31it/s] 

4323


In [6]:
X = []
Y = []

for features, labels in tqdm(data):
  X.append(features)
  Y.append(labels)

100%|██████████| 4323/4323 [00:00<00:00, 1228204.04it/s]


In [7]:
# num_classes = Y['labels'].nunique()
# print("Total number of classes:", num_classes)

In [8]:
type(X)

list

In [9]:
x = np.array(X)
y = np.array(Y)

In [10]:
np.shape(X)

(4323, 180, 180, 3)

In [11]:
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3, random_state=10)

**Applying CNN**

In [19]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size = (5,5), padding = 'Same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=196, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=328, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=912, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(228, activation='relu'))
model.add(Dense(196, activation='relu'))
model.add(Dense(5, activation='softmax'))



In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
S=time.time()
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))
E=time.time()

Epoch 1/10
95/95 [==============================] - 18s 141ms/step - loss: 7.0809 - accuracy: 0.2832 - val_loss: 1.6951 - val_accuracy: 0.2552
Epoch 2/10
95/95 [==============================] - 11s 115ms/step - loss: 1.2321 - accuracy: 0.4656 - val_loss: 1.3752 - val_accuracy: 0.4495
Epoch 3/10
95/95 [==============================] - 11s 115ms/step - loss: 1.0891 - accuracy: 0.5532 - val_loss: 0.9900 - val_accuracy: 0.6060
Epoch 4/10
95/95 [==============================] - 11s 116ms/step - loss: 0.9890 - accuracy: 0.6120 - val_loss: 1.0133 - val_accuracy: 0.6006
Epoch 5/10
95/95 [==============================] - 11s 114ms/step - loss: 0.8864 - accuracy: 0.6646 - val_loss: 0.9722 - val_accuracy: 0.6199
Epoch 6/10
95/95 [==============================] - 11s 116ms/step - loss: 0.8511 - accuracy: 0.6778 - val_loss: 0.8603 - val_accuracy: 0.6746
Epoch 7/10
95/95 [==============================] - 11s 114ms/step - loss: 0.7477 - accuracy: 0.7221 - val_loss: 1.0260 - val_accuracy: 0.6638

In [15]:
test_loss, test_acc = model.evaluate(x_test, y_test)
#joblib.dump(cnnmodel, 'cnn_model.joblib')
print(f"Test accuracy: {test_acc}")

41/41 [==============================] - 1s 27ms/step - loss: 1.1205 - accuracy: 0.6746
Test accuracy: 0.6746337413787842


|       Models       | Train Acc | Test Acc |
|:------------------:|:---------:|:--------:|
|         ANN        |   43.23   |   30.45  |
| ANN  (PCA(n=1000)) |   99.04   |   47.26  |
|  ANN  (PCA(n=300)) |   86.32   |   41.25  |
|         CNN        |   80.60   |   69.08  |